In [175]:
# FASTTEXT LANGUAGE detection 
import fasttext as ft

# Load the pretrained model
ft_model = ft.load_model("lid.176.ftz")

def fasttext_language_predict(text, model = ft_model):

  text = text.replace('\n', " ")
  prediction = model.predict([text])

  return prediction

In [176]:
def fix_languages():
    # This function will correct the languages from the .txt files 
    # It switches the NL_ and FR_ .txt files when needed
    #
    # It also checks for incorrect split documents 
    # It removes their .txt files from the /processed/data folder
    # and copies these .pdfs to /unprocessed_data/removed_from_processed
    #
    # To be executed after the split and process_docs_cloud

    import os
    from os import listdir
    import time
    import sys
    import tempfile

    erroruous_split_files=[]
    switched_docs=[]

    input_language='NL'
    opposite_language='FR'

    input_path=os.path.join('../processed_data',input_language)
    opposite_path=os.path.join('../processed_data',opposite_language)

    files_in_path=listdir(input_path)
    for file in files_in_path:
        filename=os.path.join(input_path,file)
        document_id=file[3:]

        with open(filename,encoding="utf-8") as f:
            text = f.read() 

        l_text=len(text) 

        db_check_input_language_1_half=fasttext_language_predict(text[:int(l_text/2)], model = ft_model)[0][0][0][-2:] 
        db_check_input_language_2_half=fasttext_language_predict(text[int(l_text/2):], model = ft_model)[0][0][0][-2:] 

        # Check if language detection is consistent throughout text
        if db_check_input_language_1_half==db_check_input_language_2_half:           

            detected_language=fasttext_language_predict(text, model = ft_model)[0][0][0][-2:]  

            # Check if language detected differs from folder language
            if detected_language.upper()!=input_language:

                with open (f'{opposite_path}/{opposite_language}_{document_id}',encoding="utf-8") as f:
                    opposite_text=f.read()


                opp_detected_language=fasttext_language_predict(opposite_text, model = ft_model)[0][0][0][-2:]  

                # Check if other file actually differs in language
                if detected_language!=opp_detected_language:

                    # Write the text to the opposite language folder

                    with open (f'{opposite_path}/{opposite_language}_{document_id}','w',encoding="utf-8") as fp:
                        fp.write(text)

                    # Write the opposite text to this language folder

                    with open (f'{input_path}/{input_language}_{document_id}','w',encoding="utf-8") as fp:
                        fp.write(opposite_text)

                    print (f"Switched document {document_id}",end='\n')
                    switched_docs.append(document_id)
                else:
                    # THESE FILES WILL BE REMOVED and added back to UNPROCESSED

                    erroruous_split_files.append(document_id)
                    filename=f'{input_path}/{input_language}_{document_id}'
                    if os.path.exists(filename):
                        os.remove(filename)
                    filename=f'{opposite_path}/{opposite_language}_{document_id}'
                    if os.path.exists(filename):
                        os.remove(filename)

        else:
            # THESE FILES WILL BE REMOVED 

            erroruous_split_files.append(document_id)
            filename=f'{input_path}/{input_language}_{document_id}'
            if os.path.exists(filename):
                os.remove(filename)
            filename=f'{opposite_path}/{opposite_language}_{document_id}'
            if os.path.exists(filename):
                os.remove(filename)
            
    print (f'\nSwitched documents : {len(switched_docs)}')
    print (f'Error documents : {len(erroruous_split_files)} files moved back to unprocessed')

In [177]:
fix_languages()

# Note : run move_unprocessed_files.ipynb after this

Switched document 100-2019-014388.txt
Switched document 100-2019-014742.txt
Switched document 100-2021-011883.txt
Switched document 100-2021-015007.txt
Switched document 100-2021-015017.txt
Switched document 100-2021-015021.txt
Switched document 100-2021-015023.txt
Switched document 10205-2022-002385.txt
..\data\102.06\10206-2020-000020.pdf
Notice: Removed document 10206-2020-000020
Switched document 10206-2020-000188.txt
Switched document 10208-2022-012916.txt
Switched document 10208-2022-012919.txt
Switched document 10602-2018-002118.txt
Switched document 10602-2018-002119.txt
Switched document 10602-2018-002120.txt
Switched document 10602-2018-012982.txt
Switched document 10602-2019-003097.txt
Switched document 10602-2019-012458.txt
Switched document 10602-2019-012459.txt
Switched document 10602-2019-012460.txt
Switched document 10602-2019-012461.txt
Switched document 10602-2019-012462.txt
Switched document 10602-2019-012463.txt
Switched document 10602-2019-012464.txt
Switched docum